# System Requirements:

Python 3.7+

The packages below: IPython, ipywidgets, numpy, pandas, bokeh

ipywidgets enabled  in jupyter notebook. This can be done on the command line with the two following commands:

```
>> jupyter nbextension install --py widgetsnbextension --user

>> jupyter nbextension enable widgetsnbextension --user --py
```
Though if you are using jupyter lab your setup must include the following extensions
```
>> jupyter labextension install @jupyter-widgets/jupyterlab-manager

>> jupyter labextension install @bokeh/jupyter_bokeh
```

# COVID19 Data

We will explore data sets from the New York Times about cases and deaths, Google data about mobility in different sectors of public life, and the aggregate mobility index from DescartesLabs

In [ ]:
import numpy as np
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Panel, Tabs
from bokeh.layouts import row
from bokeh.io import output_notebook, push_notebook
output_notebook()

Import New York Times Data by US state and county

In [ ]:
raw_df = pd.read_csv("http://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv",header=0, sep=",")

In [ ]:
df = raw_df.copy()
df = df.query('fips>0')

# df['date_processed'] = pd.to_datetime(df['date'].values)
# df['date_processed'] = (df['date_processed'] - pd.Timestamp('2020-03-01')) / np.timedelta64(1, 'D')
# df = df.query('date_processed>=0')

# Imposing a clean index
df['date'] = pd.to_datetime(df['date'])
df.set_index(["state","county","fips","date"],inplace=True)
df.sort_index(inplace=True)

df.head()

Import Google Mobility Data

In [ ]:
gg_cols = ["country_region_code","sub_region_1","sub_region_2","date","retail_and_recreation_percent_change_from_baseline",\
           "grocery_and_pharmacy_percent_change_from_baseline","parks_percent_change_from_baseline",\
           "transit_stations_percent_change_from_baseline","workplaces_percent_change_from_baseline",\
           "residential_percent_change_from_baseline"]

gg_df = pd.read_csv('https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?',
                    usecols=gg_cols,low_memory=False,parse_dates=['date'])
locs=["Retail & recreation","Grocery & pharmacy","Parks","Transit stations","Workplaces","Residential"]
gg_df.columns = ["country","state","county","date"]+locs
gg_df.head()

In [ ]:
# Fixing counties
gg_df["county"]=gg_df["county"].str.replace(' County', '')

def correct_county(row):
    if(row.county=='New York'):
        return "New York City"
    else:
        return row.county
gg_df['county']=gg_df.apply(lambda row: correct_county(row), axis=1)

# Formatting time
gg_df['date'] = pd.to_datetime(gg_df['date'])

# Restrict to the US states
j_df=gg_df[gg_df["country"]=="US"][pd.notna(gg_df["state"])][pd.notna(gg_df["county"])].copy()
del j_df["country"]

# Sorting joinable dataset
j_df=j_df.set_index(["state","county","date"]).sort_values(["state","county","date"])

# Indexing as, and joining with main dataset
df=df.join(j_df)

# Handling missing data
gg_df=gg_df[gg_df["country"]=="US"][pd.notna(gg_df["state"])].set_index(["state","date"]).sort_values(["state","date"])

for state, state_df in df.groupby(level='state'):
    for county, county_df in state_df.groupby(level='county'):
        for loc in locs:
            c_df=county_df[loc]
            # If not enough data points, go fetch state data 
       #     if np.sum(c_df.count())<c_df.size/10:
       #         c_df=pd.merge(c_df,gg_df[pd.isna(gg_df["county"])].loc[state][loc],how="left", \
       #                       left_index=True,right_index=True,suffixes=('_c',''))[loc]
            df.loc[(state,county),loc]=c_df.interpolate().values
    print(state, "done !")

df.head()

In [ ]:
##download and integrate Descartes Labs mobility data
des_df = pd.read_csv('https://raw.githubusercontent.com/descarteslabs/DL-COVID-19/master/DL-us-mobility-daterow.csv', \
                     usecols=['date','admin1','admin2','fips','m50_index'])
des_df.columns = ["date","state","county","fips","m50_index"]

# Correct DC
def correct_state(row):
    if(row.state=='Washington, D.C.'):
        return "District of Columbia"
    else:
        return row.state
des_df['state']=des_df.apply(lambda row: correct_state(row), axis=1)

# Copy to join as before
j_df = des_df.copy()
del j_df["state"],j_df["county"]

# Formatting time
j_df["date"] = pd.to_datetime(j_df["date"])

# Joining together the two datasets
j_df=j_df.set_index(["fips","date"]).sort_values(["fips","date"])
df=df.reset_index().sort_values(["fips","date"])
df=df.join(j_df,on=["fips","date"])

# Reindexing everything for smooth loop
des_df=des_df.set_index(["state","fips","date"]).sort_values(["state","fips","date"])
df=df.set_index(["state","fips","date"]).sort_values(["state","fips","date"])

# RIP performance
for state, state_df in df.groupby(level='state'):
    for fips, fips_df in state_df.groupby(level='fips'):
        c_df=fips_df["m50_index"]        
        # If not enough data points, go fetch state data 
        #if np.sum(c_df.count())<c_df.size/10:
        #    c_df=pd.merge(c_df,des_df[pd.isna(des_df["county"])].loc[state]["m50_index"],how="left",
        #                  left_index=True,right_index=True,suffixes=('_c',''))["m50_index"]
        df.loc[(state,fips),"m50_index"]=c_df.interpolate().values
    print(state, "done !")
df.head()

# Interactive plot
Next we plot this data aggregated by state to be able to look for trends and correlations in the data

In [ ]:
#start with national totals
initial_df = df.loc[:,'cases':'deaths'].groupby(['date']).aggregate(np.sum).reset_index()

initial_data = {'date' : initial_df['date'],
            'metric_1' : initial_df['cases'],
            'metric_2' : initial_df['deaths']} 
        
source = ColumnDataSource(initial_data)
#print(source.data)

#create initial plots
#linear metric 1
p1 = figure(title='COVID-19 data', x_axis_label='date', y_axis_label='Cumulative Cases',\
           plot_width=400, plot_height=300, x_axis_type="datetime", y_axis_type='linear')
line_p1 = p1.line(x='date', y='metric_1', source=source)

panel_p1 = Panel(child=p1, title='linear')

#log metric 1
p2 = figure(title='COVID-19 data', x_axis_label='date', y_axis_label='Cumulative Cases',\
           plot_width=400, plot_height=300, x_axis_type="datetime", y_axis_type='log')

line_p2 = p2.line(x='date', y='metric_1', source=source)

panel_p2 = Panel(child=p2, title='log')

#panel metric 1
panels_p = [panel_p1, panel_p2]

#linear metric 2
q1 = figure(title='COVID-19 data', x_axis_label='date', y_axis_label='Cumulative Cases',\
           plot_width=400, plot_height=300, x_axis_type="datetime", y_axis_type='linear')

line_q1 = q1.line(x='date', y='metric_2', source=source)

panel_q1 = Panel(child=q1, title='linear')

#log metric 2
q2 = figure(title='COVID-19 data', x_axis_label='date', y_axis_label='Cumulative Cases',\
           plot_width=400, plot_height=300, x_axis_type="datetime", y_axis_type='log')

line_q2 = q2.line(x='date', y='metric_2', source=source)

panel_q2 = Panel(child=q2, title='log')

#panel metric 2
panels_q = [panel_q1, panel_q2]
    
tabs_p = Tabs(tabs=panels_p)
tabs_q = Tabs(tabs=panels_q)

show(row(tabs_p,tabs_q), notebook_handle=True)

def update_graph(level='national', state='California', county='Alameda',\
                 metric_1='cases', metric_2='deaths', \
                 method_1='cumulative',  method_2='cumulative', \
                 averaging=1, window=None, y_scale='linear', correlation=False, cobweb=False):    

    #update county list dynamically
    def update_counties(_):
        counties = [e for e in df.loc[state_ls.value].reset_index()['county'].unique()]
        county_ls.options = counties
        county_ls.value = counties[0]
        county = counties[0]
    
    def get_data(level, state, metric, method, averaging, window):
        #fetch the data by national/state/county level
        if level == 'national':
            get_data = df[metric].groupby(['date']).aggregate(np.sum).reset_index()
        if level == 'state':
            get_data = df.loc[state, metric].groupby(['date']).aggregate(np.sum).reset_index()
        if level == 'county':
            get_data = df.loc[(state, county), metric].groupby(['date']).aggregate(np.sum).reset_index()
        
        #apply averaging / smoothing
        get_data[metric] = get_data[metric].rolling(window=averaging, win_type=window, center=False).mean()#std=averaging)
        
        #apply edits
        if method == 'difference':
            get_data[metric] = get_data[metric].diff(periods=1).dropna()
        if method == 'percent':
            get_data[metric] = get_data[metric].pct_change(periods=1).dropna()
        
        return get_data
    
    state_ls.observe(update_counties)
    
    data_metric_1 = get_data(level, state, metric_1, method_1, averaging, window)
    
    data_metric_2 = get_data(level, state, metric_2, method_2, averaging, window)
    
    #find length of new data
    length_patch = len(data_metric_1[metric_1])
    #print(length_patch)
    
    #create updated data
    stream = {'date' : data_metric_1['date'],
             'metric_1' : data_metric_1[metric_1],
             'metric_2' : data_metric_2[metric_2]}
    
    #replace existing data with new data
    source.stream(stream, rollover=length_patch)  
#     print(source.data)

    def title(level, state, county, metric):
        if level == 'national':
            return f'COVID-19 Data: {level} {metric}'
        if level == 'state':
            return f'COVID-19 Data: {state} {metric}'
        if level == 'county':
            return f'COVID-19 Data: {county} {level} {metric}'
        
    #replace chart titles
    p1.title.text = title(level,state,county,metric_1)
    p1.yaxis.axis_label = f'{method_1} {metric_1}' 
    
    p2.title.text = title(level,state,county,metric_1)
    p2.yaxis.axis_label = f'{method_1} {metric_1}' 
    
    q1.title.text = title(level,state,county,metric_2)
    q1.yaxis.axis_label = f'{method_2} {metric_2}' 
    
    q2.title.text = title(level,state,county,metric_2)
    q2.yaxis.axis_label = f'{method_2} {metric_2}' 
    
    if correlation == True:
        corr_val = data_metric_1[metric_1].corr(data_metric_2[metric_2])
        print(f'Correlation {corr_val}')
        
#     if cobweb == True:
#         p.renderers.clear()
#         step_p = p.step(x='metric_1', y='metric_1', source=source) 
#     else:
#         p.renderers.clear()
#         line_p = p.line(x='date', y='metric_1', source=source)
    
    push_notebook()
#     print()

#create widgets
level = widgets.Dropdown(options=['national', 'state', 'county'], value='national', description='level')

state_ls = widgets.Dropdown(options=[state for state in df.index.levels[0].values], value='California', description='state')

county_ls = widgets.Dropdown(options=[county for county in df.loc[state_ls.value].reset_index()['county'].unique()], value='Alameda', description='county')

metric_1 = widgets.Dropdown(options=[col for col in df.columns], value='cases', description='metric_1')

metric_2 = widgets.Dropdown(options=[col for col in df.columns], value='deaths', description='metric_2')

method_1 = widgets.Dropdown(options=['cumulative', 'difference', 'percent'], value='cumulative', description='method_1')

method_2 = widgets.Dropdown(options=['cumulative', 'difference', 'percent'], value='cumulative', description='method_2')

averaging = widgets.Dropdown(options=[k for k in range(1,10)], value=1, description='averaging')

window = widgets.Dropdown(options=[None, 'triang', 'hamming', 'gaussian'], value=None, description='window')

correlation = widgets.Checkbox(value=False, description='correlation' )

#cobweb = widgets.Checkbox(value=False, description='cobweb' )

#build widget UI
widget_dict = {'level' : level, 'state' : state_ls, 'county' : county_ls ,\
               'metric_1' : metric_1, 'metric_2' : metric_2, \
               'method_1' : method_1, 'method_2' : method_2, \
               'averaging' : averaging,'window' : window, 'correlation': correlation} #, 'cobweb' : cobweb}

col1 = widgets.VBox([level, state_ls, county_ls])
col2 = widgets.VBox([metric_1, method_1])
col3 = widgets.VBox([metric_2, method_2])
col4 = widgets.VBox([averaging, window, correlation])

ui = widgets.VBox([widgets.HBox([col1,col4]),widgets.HBox([col2,col3])])

out = widgets.interactive_output(update_graph, widget_dict)

display(ui, out)

This plot has several data manipulations. First, you can look at the data by state. Then you can control two plots. Each plot can show you a different metric in that state. For each metric, you can look at the value for that day (cumulative), the difference of the value on that day and the previous day (difference), and the percent change of the value on that day from the value the previous day (percent). The data can also be smoothed by using the averaging widget. It averages over the specified number of points, using the window (uniform, gaussian, etc) you can specify. The correlation between the two data sets plotted can also be computed, which may be more useful for comparing increases in cases to mobility, or the metrics you are interested in (warning: large correlation is not a sign of a causal relationship). Lastly the cobweb widget enables you to change how you plot the first graph so that it plots the number of cases today versus the number of cases tomorrow (interesting if you choose the metric 'cases' and the method 'difference' and compare different states: States that have 'flattened the curve' will show a line that walks up, then back down). All the data manipulations are performed using in-built pandas methods.

It would be desireable to compare data between states and to get additional data about population density, Lockdown implementation, prevalence of underlying health conditions, and COVID testing so that more meaningful conclusions could be made. Unfortunately we do not have enough time to explore all of these opportunities.

Credits: 
Geoffrey wrote all the code for merging data sets.
Lorenzo wrote all the plotting code to integrate the data with all the data computations. 